# Calculo das Proficiencias Individuais

In [109]:
proficiencia = {}
porc_vit = {}

In [110]:
import pandas as pd
import numpy as np
import random as rd
from math import ceil,floor,factorial
from scipy.optimize import minimize
Times = pd.read_excel('TIMES.xlsx')
Times.set_index('Número', inplace = True)
Times.fillna('NA', inplace = True)
Times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(Times.Sigla))), index=Times.index)
Times_msi = ['RNG','KZ','FW','FNC','TL','EVS','SUP','GMB','KBM','R7','KLG','DW','ASC','PGM']
Times_mundial_inicio = ['EDG','INF','DW','G2','SUP','ASC','C9','DFM','KBM','GRX','GMB','KLG','AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100']
Times_mundial_grupos = ['AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100','EDG','G2','C9','GRX']
Times_mundial_quartas = ['RNG','KT','FNC','IG','EDG','G2','C9','AFS']
ACERTOS=0
JOGOS_TOTAIS = 0

In [111]:
def arrumadados(sheet,guia,vero = False):
    arquivo = pd.ExcelFile(sheet)
    liga = pd.read_excel(arquivo,guia-1)
    if not vero:
        liga = liga[3:]
    liga.fillna(0,inplace = True)
    liga.replace(0.7,0.8, inplace = True)
    return liga

In [112]:
liga_time_reg = []
#Declarando as tabelas de jogos de cada regiao
cblol_2019_1 = arrumadados('CBLOL_BRASIL_2019.xlsx',1,True)
a = list(cblol_2019_1.Winner)
a.extend(list(cblol_2019_1.Loser))
times_cblol_2019 = set(a)
liga_time_reg.append((cblol_2019_1,times_cblol_2019,'BR'))

lec_2019_1 = arrumadados('LEC_EUROPA_2019.xlsx',1,True)
a = list(lec_2019_1.Winner)
a.extend(list(lec_2019_1.Loser))
times_lec_2019 = set(a)
liga_time_reg.append((lec_2019_1,times_lec_2019,'EU'))

lcs_2019_1 = arrumadados('LCS_USA_2019.xlsx',1,True)
lcs_2019_1.replace('100',100, inplace = True)
a = list(lcs_2019_1.Winner)
a.extend(list(lcs_2019_1.Loser))
times_lcs_2019 = set(a)
liga_time_reg.append((lcs_2019_1,times_lcs_2019,'NA'))

lck_2019_1 = arrumadados('LCK_COREIA_2019.xlsx',1,True)
a = list(lck_2019_1.Winner)
a.extend(list(lck_2019_1.Loser))
times_lck_2019 = set(a)
liga_time_reg.append((lck_2019_1,times_lck_2019,'KR'))

lpl_2019_1 = arrumadados('LPL_CHINA_2019.xlsx',1,True)
a = list(lpl_2019_1.Winner)
a.extend(list(lpl_2019_1.Loser))
times_lpl_2019 = set(a)
liga_time_reg.append((lpl_2019_1,times_lpl_2019,'CN'))

lms_2019_1 = arrumadados('LMS_TWHKMA_2019.xlsx',1,True)
a = list(lms_2019_1.Winner)
a.extend(list(lms_2019_1.Loser))
times_lms_2019 = set(a)
liga_time_reg.append((lms_2019_1,times_lms_2019,'TWHKMA'))

vcs_2019_1 = arrumadados('VCS_VIETNAM_2019.xlsx',1,True)
a = list(vcs_2019_1.Winner)
a.extend(list(vcs_2019_1.Loser))
times_vcs_2019 = set(a)
liga_time_reg.append((vcs_2019_1,times_vcs_2019,'VN'))

lcl_2019_1 = arrumadados('LCL_CIS_2019.xlsx',1,True)
a = list(lcl_2019_1.Winner)
a.extend(list(lcl_2019_1.Loser))
times_lcl_2019 = set(a)
liga_time_reg.append((lcl_2019_1,times_lcl_2019,'CIS'))

lla_2019_1 = arrumadados('LLA_LA_2019.xlsx',1,True)
a = list(lla_2019_1.Winner)
a.extend(list(lla_2019_1.Loser))
times_lla_2019 = set(a)
liga_time_reg.append((lla_2019_1,times_lla_2019,'LA'))

tcl_2019_1 = arrumadados('TCL_TURQUIA_2019.xlsx',1,True)
a = list(tcl_2019_1.Winner)
a.extend(list(tcl_2019_1.Loser))
times_tcl_2019 = set(a)
liga_time_reg.append((tcl_2019_1,times_tcl_2019,'TRK'))

ljl_2019_1 = arrumadados('LJL_JAPAN_2019.xlsx',1,True)
a = list(ljl_2019_1.Winner)
a.extend(list(ljl_2019_1.Loser))
times_ljl_2019 = set(a)
liga_time_reg.append((ljl_2019_1,times_ljl_2019,'JPN'))

opl_2019_1 = arrumadados('OPL_OCEANIA_2019.xlsx',1,True)
a = list(opl_2019_1.Winner)
a.extend(list(opl_2019_1.Loser))
times_opl_2019 = set(a)
liga_time_reg.append((opl_2019_1,times_opl_2019,'OCEANIA'))

lst_2019_1 = arrumadados('LST_SEA_2019.xlsx',1,True)
a = list(lst_2019_1.Winner)
a.extend(list(lst_2019_1.Loser))
times_lst_2019 = set(a)
liga_time_reg.append((lst_2019_1,times_lst_2019,'SEA'))

In [113]:
def liga_usada(liga,times,sigla):
    global x0,b,bnds
    x0 = list(np.random.uniform(0.5,2.5,len(times)))
    b = (0.001,10.0)
    bnds = tuple([b]*len(times))
    return (liga,times,sigla)

def vero(x):
    y = 0
    proficiencia[sigla] = {}
    for time in times:
        proficiencia[sigla][time] = x[y]
        y+=1
    multiplicatorio = 1
    #Defino a constante de esquecimento k, obtida experimentalmente
    k = 0.075
    for game in range (0,liga.shape[0]):
        w = liga.iloc[game].Week
        pv = proficiencia[sigla][liga.iloc[game].Winner]
        pl = proficiencia[sigla][liga.iloc[game].Loser]
        
        #Diferenciando play-offs de fase regular
        if w>0:
            cte = 1 - (max(list(liga.Week)) - w)*k
        else:
            cte = 1+k
        multiplicatorio *= (pv/(pv+pl))**cte
    
    return -np.log(multiplicatorio)

def probabilidade(A,B,n):
    siglaA = list(Times[Times.Sigla==A]['Região'])[0]
    siglaB = list(Times[Times.Sigla==B]['Região'])[0]
    A_list = []
    B_list = []
    i,k = 0,floor(n/2)
    x = proficiencia[siglaA][A]/(proficiencia[siglaA][A]+proficiencia[siglaB][B])
    print("         {} vs {}".format(A,B))
    while (ceil(n/2)+i) <= n:
        if i == 0:
            A_list.append(x**(ceil(n/2))*(1-x)**(i))
            B_list.append(x**(i)*(1-x)**(ceil(n/2)))
        else:
            p = factorial(k)/(factorial(i)*factorial(k-i)) 
            A_list.append(p*x**(ceil(n/2))*(1-x)**(i))
            B_list.append(p*x**(i)*(1-x)**(ceil(n/2)))
        i+=1
        k+=1
    if n==1:
        print('{} tem {:.2f} % de chance de vitória'.format(A,100*sum(A_list)))
        print('Enquanto {} tem {:.2f} %.'.format(B,100*sum(B_list)))
    else:    
        print('{} tem {:.2f} % de chance de vitória.Sendo:'.format(A,100*sum(A_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*A_list[b],ceil(n/2),b))
        print('Enquanto {} tem {:.2f} %. Sendo:'.format(B,100*sum(B_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*B_list[b],ceil(n/2),b))
    
def restricao1(x):
    return x[0]-1        

def restricao2(x):
    return sum(x)-5

con1 = {'type':'eq','fun':restricao1}
con2 = {'type':'eq','fun':restricao2}
cons = [con2]

In [18]:
#Para calcular a proficiencia de uma liga apenas
liga,times,sigla = liga_usada(lst_2019_1,times_lst_2019,'SEA')
sol = minimize(vero,x0,method = 'SLSQP',constraints = cons,bounds = bnds)
proficiencia[sigla]

{'CAP': 0.0010000000001488796,
 'EMP': 0.05696085581639094,
 'LYB': 0.001,
 'MEGA': 4.941039159085451}

In [114]:
#Calculando todas as proficiencias de todos os times de todas as ligas
for trio in liga_time_reg:    
    liga,times,sigla = liga_usada(trio[0],trio[1],trio[2])
    sol = minimize(vero,x0,method = 'SLSQP',constraints = cons,bounds = bnds)

In [115]:
#Pegando os dados de porcentagem de vitoria de todos os times
for trio in liga_time_reg:    
    liga,times,sigla = liga_usada(trio[0],trio[1],trio[2])
    for team in times:
        a = list(liga.Winner)
        a.extend(list(liga.Loser))
        porc_vit[team] = list(liga.Winner).count(team)/a.count(team)

#### Modelo com constante de esquecimento melhor em 66,1 % dos casos

### Previsoes da Liga Coreana
####  Taxa de Acerto na Fase de Grupos: 25/32   78,13%
#### Taxa de Acerto em Play-Offs : 3/4    75%

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Chinesa
####  Taxa de Acerto na Fase de Grupos: 38/48   79,17%
#### Taxa de Acerto em Play-Offs : 4/8 50%

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Norte Americana
####  Taxa de Acerto na Fase de Grupos: 23/31    74,19%
#### Taxa de Acerto em Play-Offs : 5/5 100%

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Europeia
#### Taxa de Acerto na Fase de Grupos: 16/26 61,54%
#### Taxa de Acerto em Play-Offs : 4/6 66,66%

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Previsoes da Liga Brasileira
#### Taxa de Acerto na Fase de Grupos: 29/36    80,55%
#### Taxa de Acerto em Play-Offs : 2/3    66,66%

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Taxa de Acerto Geral na Fase de Grupos: 131/173 75,72%
### Taxa de Acerto Geral em Play-Offs: 24/34 70,59%
### Taxa de Acerto Geral Final: 74,88%

# Calculo da proficiencia relativa entre as regioes

In [116]:
#Trocando os nomes de duas regioes que se juntaram e se tornaram uma so
Times.replace('LAS','LA',inplace = True)
Times.replace('LAN','LA',inplace = True)
#Criando um dicionario com todas as regioes
regions = {}
for reg in set(Times.Região): 
    regions[reg] = list(Times.Sigla[Times.Região == reg])
    
#Criando um com todos os mundiais e importando os respectivos jogos
    mundial = {}
for ano in range (2011,2019):
    mundial[ano] = arrumadados("MUNDIAIS.xlsx",ano-2010,True)

#Criando um dataframe com as regiões e suas proficiências em cada ano
Proficiencias_Relativas = pd.DataFrame(data = None, index = list(set(Times.Região)),columns = [2011,2012,2013,2014,2015,2016,2017,2018,2019])


In [117]:
#Criando uma função que adiciona a proficiência relativa de cada região no dataframe
def adicione(ano):
    m = min(proficiencia.values())
    M = max(proficiencia.values())
    for reg in set(Times.Região):
        if reg in list(proficiencia.keys()):
            Proficiencias_Relativas.loc[[reg],[ano]] = (proficiencia[reg]-m)/(M-m)
            
#Modificando a funcao original pra que se adeque ao objetivo proposto
def vero_reg(x):
    y = 0
    for time in times:
        proficiencia_reg[time] = x[y]
        y+=1
    multiplicatorio = 1
    for ano in range(2011,2019):
        cship = mundial[ano]
        coef = 1 - (2018-ano)/8
        for game in range (0,int(cship.shape[0])):
            pv = proficiencia_reg[cship.iloc[game].Winner]
            pl = proficiencia_reg[cship.iloc[game].Loser]
            multiplicatorio *= (pv/(pv+pl))**(coef)
    
    return -np.log(multiplicatorio)

def probabilidadeR(A,B):
    x = proficiencia_reg[A]/(proficiencia_reg[A]+proficiencia_reg[B])
    print("         {} vs {}".format(A,B))
    print("{} tem {:.2f} % de chance de vitória".format(A,100*x))
    print("Enquanto {} tem {:.2f} %".format(B,100*(1-x)))

In [118]:
for camp in mundial.values():
    for reg in set(Times.Região):
        for linha in range(0,camp.shape[0]):
            if camp.iloc[linha].Winner in regions[reg]:
                camp.iloc[linha].Winner = reg
            if camp.iloc[linha].Loser in regions[reg]:
                camp.iloc[linha].Loser = reg

In [119]:
proficiencia_reg = {}
cons = con1
liga,times,sigla = liga_usada(lck_2019_1,list(set(Times.Região)),'MUND')
sol = minimize(vero_reg,x0,method = 'SLSQP',constraints = cons,bounds = bnds)
proficiencia_reg

{'BR': 0.4966737276332403,
 'CIS': 3.0753858063045345,
 'CN': 5.813425195257271,
 'EU': 4.482059043516012,
 'JPN': 0.2992857402061734,
 'KR': 10.0,
 'LA': 1.0,
 'NA': 3.576430271854339,
 'OCEANIA': 0.350498688454531,
 'SEA': 0.07019218585635462,
 'TRK': 1.1874555150210941,
 'TWHKMA': 2.0987945021911747,
 'VN': 1.7233568542067375}

In [120]:
sorted(proficiencia_reg.items(), key=lambda x: x[1])

[('SEA', 0.07019218585635462),
 ('JPN', 0.2992857402061734),
 ('OCEANIA', 0.350498688454531),
 ('BR', 0.4966737276332403),
 ('LA', 1.0),
 ('TRK', 1.1874555150210941),
 ('VN', 1.7233568542067375),
 ('TWHKMA', 2.0987945021911747),
 ('CIS', 3.0753858063045345),
 ('NA', 3.576430271854339),
 ('EU', 4.482059043516012),
 ('CN', 5.813425195257271),
 ('KR', 10.0)]

In [121]:
#Fazendo a conversao de medida entre as escalas individuais de cada regiao para a escala global
#Assim, podemos fazer previsoes entre times de outras regioes
for trio in liga_time_reg:    
    sigla = trio[2]    
    top3 = list(proficiencia[sigla].values())
    top3.sort(reverse = True)
    top3 = top3[0:3]
    harm = 3/(1/top3[0] + 1/top3[1] + 1/top3[2])
    p_relativa = proficiencia_reg[sigla]
    proficiencia[sigla].update((x,y*p_relativa/harm) for x,y in proficiencia[sigla].items())

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [122]:
## Criando um dataframe com os times ativos em 2019
listadesiglas = []
for i in range (0,len(proficiencia.keys())):
    for sigla in list(list(proficiencia.values())[i].keys()):
        listadesiglas.append(sigla)
Times_2019 = Times[Times.Sigla.isin(listadesiglas)]
del Times_2019['Força Inicial']
del Times_2019['Força atualizada']
lista2 = []

for time in Times_2019.Sigla:
    lista2.append(proficiencia[list(Times_2019[Times_2019.Sigla==time]['Região'])[0]][time])
    Times_2019.at[Times_2019[Times_2019.Sigla==time].index, 'Porcentagem de Vitorias'] = porc_vit[time]
            
Times_2019['Proficiencia'] = pd.Series(lista2, index=Times_2019.index)
Times_2019['Porcentagem de Vitorias'] = Times_2019['Porcentagem de Vitorias'].apply(lambda x:int(10000*x))
Times_2019['Porcentagem de Vitorias'] = Times_2019['Porcentagem de Vitorias'].apply(lambda x:x/100)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [125]:
Times_2019[Times_2019['Região']=='KR'].sort_values(by =['Proficiencia'], ascending = False).round(3)

,Sigla,Região,Porcentagem de Vitorias,Proficiencia
Número,,,,
7,SKT,KR,74.00,14.539
5,KZ,KR,66.66,9.383
4,GRF,KR,73.80,8.023
19,DWG,KR,56.25,4.654
18,SB,KR,61.70,4.478
6,HLE,KR,48.78,2.883
2,GEN,KR,36.36,1.815
3,AFS,KR,32.55,1.490
1,KT,KR,30.95,1.259


In [131]:
probabilidade('G2','TL',3)

         G2 vs TL
G2 tem 69.90 % de chance de vitória.Sendo:
40.46 % a de ser 2-0
29.45 % a de ser 2-1
Enquanto TL tem 30.10 %. Sendo:
13.25 % a de ser 2-0
16.85 % a de ser 2-1
